In [4]:
import pandas as pd
import numpy as np
import os
import datetime
import win32com.client
import time


In [5]:
inicio = time.time()

LECTURA DE DATAFRAMES

In [6]:

#ME2L POR SEPARADO
me2l_df = pd.read_excel('me2l r3.XLSX', dtype={'Posición':'str', 'Documento compras':'str'})




In [7]:
me2l_df = me2l_df.drop(0)

In [8]:
#BASE PLANIFICABLE
base = pd.read_excel("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras Maestros/Base Planificable/2024-03 Base Planificable/Base Marzo OEM&AXS R3.xlsx", header=1)
base_df_prov = base[['Proveedor', 'Pais (Proveedor)']]
base_df = base[['Material','Texto breve de material','NomSector_actual','TIPO','Corresponde','Proveedor', 'Pais (Proveedor)']]

#base_df = base_df[['Material','Texto breve de material','NomSector_actual','TIPO','Corresponde']]

In [9]:
base

,Material,Texto breve de material,Part Number,Material S4,NomSector_actual,TIPO,Proveedor,Nombre Proveedor,Proveedor regular,Pais (Proveedor),...,Altura,Reg.info de compras,Sector_MU,MOQ x Factor,Precio,Moneda,Marca,Modelo,Costo un,Esta en FC?
0,8383179P00000,GOMA PTA TRAS DER,8383179P00000,0,Suzuki,OEM,0,0,NaN,0,...,0.001,0,1,NaN,0.0,0,NaN,Sin Info,NaN,0
1,"""82510M84P01000\n""","""CHAPA MALETA\n""","""82510M84P01000\n""",0,Suzuki,OEM,0,0,NaN,0,...,0.001,0,1,NaN,0.0,0,NaN,Sin Info,NaN,0
2,"""B3011282701\n""","""OPACADOR PTD 3\n""","""B3011282701\n""",0,Changan,OEM,0,0,NaN,0,...,0.001,0,6,NaN,0.0,0,NaN,Sin Info,NaN,0
3,0.02.242470ATG,JUNTA DE ESTANQUEIDAD E40X,0.02.242470ATG,ATG100834,JAC Cars,OEM,40001373,ANHUI JIANGHUAI AUTOMOBIL,X,CN,...,40.000,5301579864,7,NaN,0.5,USD,NaN,JAC J3,NaN,0
4,0.02.242756ATG,TUBERIA DE REFRIGERANTE 1,0.02.242756ATG,ATG100827,JAC Cars,OEM,40001373,ANHUI JIANGHUAI AUTOMOBIL,X,CN,...,39.000,5301579676,7,NaN,0.5,USD,NaN,JAC J3,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261190,ZZJ318840,(SALE) SENSOR DE TEMPERATURA,ZZJ318840,1036470,Mazda,OEM,40000139,MAZDA MOTOR CORPORATION,X,JP,...,7.000,5301566937,2,NaN,1579.0,JPY,NaN,MZD CX-9,NaN,1
261191,ZZL151155,PORTA AMPOLLETA,ZZL151155,ZZL1511550000,Mazda,OEM,0,0,NaN,0,...,8.500,0,2,NaN,0.0,0,NaN,Sin Info,NaN,0
261192,ZZL551155,PORTA AMPOLLETA,ZZL551155,ZZL5511550000,Mazda,OEM,0,0,NaN,0,...,5.000,0,2,NaN,0.0,0,NaN,Sin Info,NaN,0
261193,ZZMA18100A,BOBINA,ZZMA18100A,548514,Mazda,OEM,40000139,MAZDA MOTOR CORPORATION,X,JP,...,20.000,5300875802,2,NaN,13375.0,JPY,NaN,MZD MPV,NaN,0


In [11]:
#CODIGOS CADENA DE REEMPLAZO
cod_actual_df = pd.read_excel('COD_ACTUAL_R3.xlsx', usecols = ['Nro_pieza_fabricante_1', 'Cod_Actual_1'])
base_df_ue = pd.merge(base_df, cod_actual_df, left_on="Material", right_on="Nro_pieza_fabricante_1", how="left")
base_df_ue['Cod_Actual_1'] = base_df_ue['Cod_Actual_1'].fillna(base_df_ue['Material'])
base_df_ue = base_df_ue[['Cod_Actual_1','Texto breve de material', 'NomSector_actual', 'TIPO','Corresponde']]
base_df_ue = base_df_ue.rename(columns={'Cod_Actual_1':'Material'})
base_df_ue = base_df_ue.drop_duplicates(subset=['Material'])

TRATAMIENTO DE DFS (OC, POSiCION)

In [12]:
print(f'Dimensiones del df1: {me2l_df.shape}')



Dimensiones del df1: (19034, 37)


UNIR DFS

In [264]:
#me2l_consolidado = pd.concat([me2l1_df,me2l2_df,me2l3_df])


In [265]:
#me2l_consolidado.reset_index(drop=True, inplace=True)

ULTIMO ESLABON A ME2L

In [13]:
me2l_df['Material'] = me2l_df['Material'].astype('str')
me2l_df['Material'] = me2l_df['Material'].apply(lambda x: x.split(".")[0])
print(f'Dimensiones de consolidado: {me2l_df.shape}')
me2l_ue = pd.merge(me2l_df, cod_actual_df, left_on="Material", right_on="Nro_pieza_fabricante_1", how="left")
me2l_ue['Cod_Actual_1'].fillna(me2l_ue['Material'], inplace=True)

me2l_ue = me2l_ue[['Documento compras', 'Posición', 'Reparto','Cl.documento compras','Tipo doc.compras',
       'Grupo de compras','Historial pedido/Docu.orden entrega', 'Fecha documento', 'Material','Cod_Actual_1', 'Texto breve','Grupo de artículos',
       'Indicador de borrado','Tipo de posición', 'Tipo de imputación', 'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda', 'Cantidad base',
       'Fecha de entrega','Hora', 'Fecha entrega estad.',  'Cantidad entrada', 'Cantidad de salida', 'Cantidad entregada',
       'Solicitud de pedido', 'Pos.solicitud pedido', 'Indicador creación',
       'Cantidad de posiciones', 'Nombre del proveedor',
       'Por entregar (cantidad)', 'Por entregar (valor)',
       'Por calcular (cantidad)', 'Por calcular (valor)'
       ]]

me2l_ue = me2l_ue.rename(columns={'Material':'Material Antiguo'})
me2l_ue = me2l_ue.rename(columns={'Cod_Actual_1':'Material'})


Dimensiones de consolidado: (19034, 37)


In [14]:
base_df_prov = base_df_prov.drop_duplicates(subset=['Proveedor'])

In [15]:
base_df_ue

,Material,Texto breve de material,NomSector_actual,TIPO,Corresponde
0,8383179P00000,GOMA PTA TRAS DER,Suzuki,OEM,0
1,"""82510M84P01000\n""","""CHAPA MALETA\n""",Suzuki,OEM,0
2,"""B3011282701\n""","""OPACADOR PTD 3\n""",Changan,OEM,0
3,0.02.242470ATG,JUNTA DE ESTANQUEIDAD E40X,JAC Cars,OEM,1
4,0.02.242756ATG,TUBERIA DE REFRIGERANTE 1,JAC Cars,OEM,1
...,...,...,...,...,...
261189,ZZJ318840,SENSOR DE TEMPERATURA,Mazda,OEM,1
261191,ZZL151155,PORTA AMPOLLETA,Mazda,OEM,0
261192,ZZL551155,PORTA AMPOLLETA,Mazda,OEM,0
261193,ZZMA18100A,BOBINA,Mazda,OEM,1


In [16]:
me2l_cruce_sector = pd.merge(me2l_ue, base_df_ue[['Material','Texto breve de material', 'NomSector_actual', 'TIPO','Corresponde']], left_on="Material", right_on="Material", how="left")
#me2l_ue.to_excel("C:/Users/lravlic/PROYECTOS DATA/PRUEBAS TRANSITO/me2l_sector.xlsx")
me2l_cruce_sector.shape
me2l_cruce_sector = me2l_cruce_sector[['Documento compras', 'Posición', 'Reparto','Cl.documento compras','Tipo doc.compras',
       'Grupo de compras','Historial pedido/Docu.orden entrega', 'Fecha documento', 'Material Antiguo','Material', 'Texto breve','Grupo de artículos',
       'Indicador de borrado','Tipo de posición', 'Tipo de imputación', 'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda', 
       'Fecha de entrega','Hora', 'Fecha entrega estad.',
       'Cantidad entrada', 'Cantidad de salida', 'Cantidad entregada',
       'Solicitud de pedido', 'Pos.solicitud pedido', 'Indicador creación',
       'Cantidad de posiciones', 'Nombre del proveedor',
       'Por entregar (cantidad)', 'Por entregar (valor)',
       'Por calcular (cantidad)', 'Por calcular (valor)', 'NomSector_actual',
       'Texto breve de material', 'TIPO', 'Corresponde']]
me2l_cruce_sector['Cod_Prov'] = me2l_cruce_sector['Nombre del proveedor'].str.split(' ', expand=True)[0]
me2l_cruce_sector = me2l_cruce_sector.merge(base_df_prov,left_on='Cod_Prov', right_on='Proveedor', how='left')
me2l_cruce_sector['Posición'] = me2l_cruce_sector['Posición'].astype('str')
me2l_cruce_sector['AUX'] = me2l_cruce_sector['Documento compras'] + me2l_cruce_sector['Posición']
me2l_cruce_sector['Origen'] = me2l_cruce_sector['Pais (Proveedor)']
me2l_cruce_sector = me2l_cruce_sector[['AUX','Documento compras', 'Posición', 'Reparto','Cl.documento compras','Tipo doc.compras',
       'Grupo de compras','Historial pedido/Docu.orden entrega', 'Fecha documento', 'Material Antiguo','Material', 'Texto breve','Grupo de artículos',
       'Indicador de borrado','Tipo de posición', 'Tipo de imputación', 'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda',
       'Fecha de entrega','Hora', 'Fecha entrega estad.',
       'Cantidad entrada', 'Cantidad de salida', 'Cantidad entregada',
       'Solicitud de pedido', 'Pos.solicitud pedido', 'Indicador creación',
       'Cantidad de posiciones', 'Nombre del proveedor',
       'Por entregar (cantidad)', 'Por entregar (valor)',
       'Por calcular (cantidad)', 'Por calcular (valor)', 'NomSector_actual','Origen', 'TIPO',  'Corresponde']]
me2l_base_tr = me2l_cruce_sector[me2l_cruce_sector['NomSector_actual'].notna()]
me2l_base_tr['Corresponde'].fillna(0, inplace=True)
me2l_base_tr['Corresponde'].replace({0:1}, inplace=True)
me2l_base_tr

C:\Users\lravlic\AppData\Local\Temp\ipykernel_13336\1006450477.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  me2l_base_tr['Corresponde'].fillna(0, inplace=True)
C:\Users\lravlic\AppData\Local\Temp\ipykernel_13336\1006450477.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  me2l_base_tr['Corresponde'].replace({0:1}, inplace=True)


,AUX,Documento compras,Posición,Reparto,Cl.documento compras,Tipo doc.compras,Grupo de compras,Historial pedido/Docu.orden entrega,Fecha documento,Material Antiguo,...,Cantidad de posiciones,Nombre del proveedor,Por entregar (cantidad),Por entregar (valor),Por calcular (cantidad),Por calcular (valor),NomSector_actual,Origen,TIPO,Corresponde
3,420000520910,4200005209,10,1.0,ZSTO,F,RR2,NaN,2024-04-09,PORTADOCCHANGANP,...,1,10001447 I.V. Fabrica de producto,1021.0,2542290.00,1021.0,2542290.00,Accesorios,CL,AXS,1.0
4,420000520920,4200005209,20,1.0,ZSTO,F,RR2,NaN,2024-04-09,PORTADOCSUZUKI,...,1,10001447 I.V. Fabrica de producto,470.0,1400600.00,470.0,1400600.00,Accesorios,CL,AXS,1.0
5,470001860210,4700018602,10,1.0,ZSPT,F,RR2,NaN,2024-04-17,PORTADCTOKYAL,...,1,10001447 I.V. Fabrica de producto,2034.0,8339400.00,2034.0,8339400.00,Accesorios,CL,AXS,1.0
6,420000521210,4200005212,10,1.0,ZSTO,F,RR2,NaN,2024-04-09,LONAMARITIMANBT,...,1,10001511 Comercial Tier Sa,6.0,705702.00,6.0,705702.00,Accesorios,CL,AXS,1.0
7,430001440010,4300014400,10,1.0,ZVOR,F,RR4,NaN,2024-03-27,NE14T0030200,...,1,10001519 JESUS PONS FRANCO Y CIA.,4.0,42000.00,4.0,42000.00,Changan,NaN,OEM,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19029,4300014891120,4300014891,120,1.0,ZVOR,F,RR9,NaN,2024-04-22,8200578089,...,1,PU02 DERCO PERU S.A.,1.0,15.92,1.0,15.92,Renault,PE,OEM,1.0
19030,4300014891130,4300014891,130,1.0,ZVOR,F,RR9,NaN,2024-04-22,497610324R,...,1,PU02 DERCO PERU S.A.,1.0,43.56,1.0,43.56,Renault,PE,OEM,1.0
19031,4300014891140,4300014891,140,1.0,ZVOR,F,RR9,NaN,2024-04-22,265105055R,...,1,PU02 DERCO PERU S.A.,1.0,14.04,1.0,14.04,Renault,PE,OEM,1.0
19032,4300014891150,4300014891,150,1.0,ZVOR,F,RR9,NaN,2024-04-22,638415794R,...,1,PU02 DERCO PERU S.A.,1.0,45.60,1.0,45.60,Renault,PE,OEM,1.0


In [270]:
me2l_base_tr.to_excel('me2l r3 sector.xlsx')

In [17]:
me2l_base_tr['Corresponde'].fillna(0, inplace=True)
me2l_base_tr['Corresponde'].replace({0:1}, inplace=True)

C:\Users\lravlic\AppData\Local\Temp\ipykernel_13336\3910453820.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  me2l_base_tr['Corresponde'].fillna(0, inplace=True)
C:\Users\lravlic\AppData\Local\Temp\ipykernel_13336\3910453820.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  me2l_base_tr['Corresponde'].replace({0:1}, inplace=True)


In [18]:
bases_oc = me2l_base_tr.groupby(['Documento compras'])['Posición'].count().sort_values(ascending=False).reset_index()

In [ ]:
bases_oc['Documento compras'].to_clipboard(header=False, index=False)

In [21]:

me2l_base_tr.to_excel("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Tubo Semanal/2024-04-23/2024-04-23 ME2L R3.xlsx")

In [23]:
SapGuiAuto = win32com.client.GetObject("SAPGUI")
application = SapGuiAuto.GetScriptingEngine
connection = application.Children(0)
session = connection.Children(0)
#now = datetime.now()

#session.findById("wnd[0]").maximize
#session.findById("wnd[0]").maximize
session.findById("wnd[0]/tbar[0]/okcd").text = "vl06if"
session.findById("wnd[0]").sendVKey(0)
session.findById("wnd[0]/usr/ctxtIT_LFDAT-LOW").text = ""
session.findById("wnd[0]/usr/ctxtIT_LFDAT-HIGH").text = ""
session.findById("wnd[0]/usr/ctxtIT_LFDAT-LOW").setFocus()
session.findById("wnd[0]/usr/ctxtIT_LFDAT-LOW").caretPosition = 0
session.findById("wnd[0]").sendVKey(0)
session.findById("wnd[0]/usr/ctxtIT_LFDAT-HIGH").setFocus()
session.findById("wnd[0]/usr/ctxtIT_LFDAT-HIGH").caretPosition = 0
session.findById("wnd[0]").sendVKey(0)
session.findById("wnd[0]/usr/btn%_IT_EBELN_%_APP_%-VALU_PUSH").press()
session.findById("wnd[1]/tbar[0]/btn[24]").press()
session.findById("wnd[1]/tbar[0]/btn[0]").press()
session.findById("wnd[1]/tbar[0]/btn[8]").press()
session.findById("wnd[0]").sendVKey(8)
session.findById("wnd[0]/tbar[1]/btn[18]").press()
session.findById("wnd[0]").sendVKey(33)
session.findById("wnd[1]/usr/lbl[1,7]").setFocus()
session.findById("wnd[1]/usr/lbl[1,7]").caretPosition = 4
session.findById("wnd[1]").sendVKey(2)
session.findById("wnd[0]").sendVKey(43)
session.findById("wnd[1]/usr/cmbG_LISTBOX").setFocus()
session.findById("wnd[1]/tbar[0]/btn[0]").press()
session.findById("wnd[1]/usr/ctxtDY_PATH").text = "C:/Users/lravlic/PROYECTOS DATA/PRUEBAS TRANSITO"
session.findById("wnd[1]/tbar[0]/btn[11]").press()
session.findById("wnd[0]").sendVKey(3)
session.findById("wnd[0]").sendVKey(3)




com_error: (-2147352567, 'Ocurrió una excepción.', (617, 'SAP Frontend Server', 'The virtual key is not enabled.', None, 0, 0), None)

In [275]:
# def close_excel_file(file_path):
#     try:
#         excel = win32com.client.Dispatch("Excel.Application")
#         workbook = excel.Workbooks.Open(file_path)
#         if workbook:
#             workbook.Close(SaveChanges=False)
#             excel.Quit()
#         else:
#             print("Failed to open the workbook.")
#     except Exception as e:
#         print("An error occurred:", e)


# close_excel_file('C:/Users/lravlic/PROYECTOS DATA/PRUEBAS TRANSITO/export.XLSX')

In [276]:
df_ee = pd.read_excel('export.XLSX', dtype={'Documento compras':'str' , 'Posición modelo':'str'})

In [277]:
monitor = pd.read_csv('C:/Users/lravlic/PROYECTOS DATA/PRUEBAS TRANSITO/sql_mon_r3.csv', dtype={'NRO_MIRO':'str','POS_OC_SAP':'str','NRO_OC_SAP':'str'})

In [278]:
monitor.dtypes

NRO_OC_SAP           object
POS_OC_SAP           object
NRO_MIRO             object
NOMBRE_PROVEEDOR     object
PROVEEDOR            object
NRO_SOLPED            int64
IND_BORRADO_OC       object
IND_BORRADO_PO       object
IND_BORRADO_SP        int64
CANT_UN_OC            int64
CANT_FACTURA        float64
CANTIDAD_BO         float64
VIA_TEXTO            object
FE_CONFIRMACION      object
FE_ATA               object
dtype: object

In [279]:
monitor['IND_BORRADO_SP'].value_counts()

IND_BORRADO_SP
0    528521
1    240334
Name: count, dtype: int64

In [280]:
monitor['NRO_OC_SAP'] = monitor['NRO_OC_SAP'].astype('str')
monitor['POS_OC_SAP'] = monitor['POS_OC_SAP'].fillna(0).astype(int)
monitor['POS_OC_SAP'] = monitor['POS_OC_SAP'].astype('str')


monitor['POS_OC_SAP']
monitor['AUX'] = monitor['NRO_OC_SAP'] + monitor['POS_OC_SAP']
monitor['NRO_MIRO'] = monitor['NRO_MIRO'].astype('str')
#revision = monitor_df.groupby('AUX')['Fe. ATA'].nunique().sort_values(ascending=False).reset_index()
tabla_1 = monitor[monitor['IND_BORRADO_OC'].isna() &
    monitor['IND_BORRADO_SP']==0 &
    monitor['IND_BORRADO_PO'].isna()].groupby(['AUX', 'NRO_MIRO']).agg({'CANT_UN_OC':'mean', 'CANT_FACTURA':'max','CANTIDAD_BO':'mean'}).reset_index()
tabla_final = tabla_1.groupby(['AUX']).agg({'CANT_UN_OC':'mean', 'CANT_FACTURA':'sum','CANTIDAD_BO':'mean'}).reset_index()
print(f'Dimension de tabla final: {tabla_final.shape}')
print('-'*40)
print(f'Total Campo OC: {tabla_final['CANT_UN_OC'].sum()}')
print(f'Total Campo Factura: {tabla_final['CANT_FACTURA'].sum()}')
print(f'Total Campo BO: {tabla_final['CANTIDAD_BO'].sum()}')

Dimension de tabla final: (470889, 4)
----------------------------------------
Total Campo OC: 7452027.0
Total Campo Factura: 6265054.444
Total Campo BO: 667688.3155238094


In [281]:
df_ee = df_ee[['Documento compras','Posición modelo', 'Cantidad entrega', 'Status mov.mcía.']]
df_ee['Documento compras'] = df_ee['Documento compras'].astype('str')
df_ee['Posición modelo'] = df_ee['Posición modelo'].astype('str')

In [282]:
df_ee

,Documento compras,Posición modelo,Cantidad entrega,Status mov.mcía.
0,4300012507,10,1,C
1,4300012510,100,1,C
2,4300012512,10,1,C
3,4200004133,60,2,C
4,4200004133,70,1,C
...,...,...,...,...
14394,4300014345,20,2,A
14395,4300014346,10,1,A
14396,4300014348,10,1,A
14397,4300014349,10,1,A


In [283]:
df_ee['AUX'] = df_ee['Documento compras'] + df_ee['Posición modelo']
df_ee.drop(columns=['Documento compras','Posición modelo'])

,Cantidad entrega,Status mov.mcía.,AUX
0,1,C,430001250710
1,1,C,4300012510100
2,1,C,430001251210
3,2,C,420000413360
4,1,C,420000413370
...,...,...,...
14394,2,A,430001434520
14395,1,A,430001434610
14396,1,A,430001434810
14397,1,A,430001434910


In [284]:
table_ee = df_ee[df_ee['Status mov.mcía.']=='C'].groupby(['AUX'])['Cantidad entrega'].sum().reset_index()

In [285]:
me2l_base_tr.to_excel('Base_+_15.xlsx')

In [286]:
me2l_base_tr.shape

(15626, 42)

In [287]:
me2l_15 = me2l_base_tr

In [288]:
monitor.drop_duplicates(subset=['AUX'], inplace=True)
monitor['FE_ATA'] = pd.to_datetime(monitor['FE_ATA'])

In [289]:
me2l_15 = me2l_15.merge(monitor[['AUX','FE_ATA']], left_on='AUX', right_on='AUX', how = 'left')
me2l_15['Fecha Est. Fact'] = np.where(me2l_15['FE_ATA'].notnull(), me2l_15['FE_ATA'] + pd.Timedelta(days=15), me2l_15['Fecha de entrega'])
me2l_15 = me2l_15.merge(tabla_final[['AUX', 'CANT_FACTURA']], left_on = 'AUX', right_on='AUX',how='left')
me2l_15['Cant. Factura'] = me2l_15['CANT_FACTURA'] * me2l_15['Corresponde']
me2l_15= me2l_15.merge(table_ee, left_on='AUX', right_on='AUX', how='left')
me2l_15['Cantidad entrega'] = me2l_15['Cantidad entrega']*me2l_15['Corresponde']
me2l_15['Qty por Entregar'] = me2l_15['Por entregar (cantidad)']*me2l_15['Corresponde']

In [290]:
#cambiar esto por la fecha del dia actual

hoy = datetime.datetime.today()
#hoy = datetime.datetime.today()

In [291]:
hoy

datetime.datetime(2024, 3, 26, 9, 33, 33, 601399)

In [292]:
me2l_15['Cantidad entrega'].fillna(0, inplace=True)

In [293]:


# Suponiendo que 'me2l_15' es tu DataFrame y 'AW4', 'BJ4' y 'AT4' son columnas en tu DataFrame
result = me2l_15['Cant. Factura'] - me2l_15['Cantidad entrega']
result = np.where(np.isnan(result), me2l_15['Qty por Entregar'], result)

me2l_15['Qty Fact Corr'] = np.where(result<0, 0, result)

In [294]:
me2l_15['Qty Fact Corr'].sum()

78588.0

In [295]:
hoy = pd.to_datetime(hoy)

In [296]:
me2l_15[ 'Fecha Est. Fact'].max()

Timestamp('2024-09-02 00:00:00')

In [297]:
conditions = [
    (me2l_15['Qty Fact Corr'] > 0) & (me2l_15['Fecha Est. Fact'] >= hoy),
    (me2l_15['Qty Fact Corr'] > 0) & (me2l_15['Fecha Est. Fact'] < hoy)
]

choices = ['Facturado No Vencido', 'Facturado Vencido']

me2l_15['Status TR Fact'] = np.select(conditions, choices, default='No Facturado')

In [298]:
me2l_15['Status TR Fact'].value_counts()

Status TR Fact
No Facturado            8472
Facturado No Vencido    5340
Facturado Vencido       1814
Name: count, dtype: int64

In [299]:
from datetime import timedelta, datetime

hoy_31 = hoy  + timedelta(days=31)

In [300]:
condition = (me2l_15['Fecha de entrega'] > hoy) & (me2l_15['Fecha de entrega'] < hoy_31)

# Apply the condition using boolean indexing and assign 'SI' or 'NO' accordingly
me2l_15['Fecha BO Mes Actual'] = np.where(condition, 'SI', 'NO')

In [301]:
me2l_15['Fecha BO Mes Actual'].value_counts()

Fecha BO Mes Actual
NO    12481
SI     3145
Name: count, dtype: int64

In [302]:
condition= (me2l_15['Fecha de entrega']< hoy)

me2l_15['TR Vencido'] = np.where(condition, 'SI','NO')

In [303]:
me2l_15['TR Vencido'].value_counts()

TR Vencido
NO    13385
SI     2241
Name: count, dtype: int64

In [304]:
conditions = [
    (me2l_15['Origen'] == 'NAC') & (me2l_15['TIPO']=='AFM'),
    (me2l_15['Fecha BO Mes Actual'] == "NO") & (me2l_15['TR Vencido'] == "NO"),
    (me2l_15['TR Vencido'] == "SI"),
]

# Define choices
choices = [
    "Replanificar +45",
    "Fecha Teórica",
    "TR Vencido",
]

# Apply np.select
me2l_15['Statur TR No FAct'] = np.select(conditions, choices, default="Replanificar +45")

In [305]:
me2l_15['Cant. Factura'].sum()

52065.0

In [306]:
#Qty BO
me2l_15['Cant. BO'] = me2l_15['Cant. Factura'].fillna(0, inplace=True)

In [307]:
#Cant Factura : Cant Fact CORR
#QTY BO: CHECK

In [308]:
condicion = (me2l_15['Fecha Est. Fact'] < hoy)

me2l_15['Atraso'] = np.where(condicion, 'SI','NO')

In [309]:
me2l_15['Atraso'].value_counts()

Atraso
NO    13322
SI     2304
Name: count, dtype: int64

In [310]:
condicion = (me2l_15['Atraso'] == 'SI')

me2l_15['Fecha Facturacion Final'] = np.where(condicion, hoy_31, me2l_15['Fecha Est. Fact'])



In [311]:
me2l_15['Fecha Facturacion Final'] = pd.to_datetime(me2l_15['Fecha Facturacion Final'])

In [312]:
me2l_15 = me2l_15.merge(monitor[['AUX', 'VIA_TEXTO']], left_on='AUX', right_on='AUX', how='left')

In [313]:
me2l_15['LT Objetivo'] = np.where(me2l_15['VIA_TEXTO'].isin(["Marítimo", "Terrestre"]), 45,
                    np.where(me2l_15['VIA_TEXTO'] == "Aéreo", 15,
                        np.where(me2l_15['VIA_TEXTO'] == "Courier", 7, 0)))

In [314]:
me2l_15 = me2l_15.merge(monitor[['AUX','FE_CONFIRMACION']], left_on='AUX',right_on='AUX', how='left')

In [315]:
me2l_15['Fecha OC'] = np.where((me2l_15['FE_CONFIRMACION'] == 0) | (me2l_15['FE_CONFIRMACION'].isna()), me2l_15['Fecha documento'], me2l_15['FE_CONFIRMACION'])

In [316]:
import datetime
today = hoy

In [317]:
me2l_15['Fecha OC'] = pd.to_datetime(me2l_15['Fecha OC'])

In [318]:
diferencia_dias = today - me2l_15['Fecha OC']

# Convertir la diferencia a días y restar el LT Objetivo
me2l_15['Dias LT BO'] = diferencia_dias.dt.days - me2l_15['LT Objetivo']

# Manejar errores potenciales, estableciéndolos en 0
me2l_15['Dias LT BO'] = me2l_15['Dias LT BO'].fillna(0)


# Calculate the difference
#me2l_15['Dias LT BO'] = (today - me2l_15['Fecha OC']).dt.days - me2l_15['LT Objetivo']

# Handling potential errors, setting them to 0
#me2l_15['Dias LT BO'] = me2l_15['Dias LT BO'].fillna(0)

In [319]:
me2l_15['Dias LT BO'].sum()

285386

In [320]:
import pandas as pd
import numpy as np

# Define conditions and choices
conditions = [
    (me2l_15['VIA_TEXTO'].isin(["Marítimo", "Terrestre"])) & (me2l_15['Dias LT BO'].between(1, 30)),
    (me2l_15['VIA_TEXTO'].isin(["Marítimo", "Terrestre"])) & (me2l_15['Dias LT BO'].between(31, 60)),
    (me2l_15['VIA_TEXTO'].isin(["Marítimo", "Terrestre"])) & (me2l_15['Dias LT BO'].between(61, 90)),
    (me2l_15['VIA_TEXTO'].isin(["Marítimo", "Terrestre"])) & (me2l_15['Dias LT BO'] > 90),
    (me2l_15['VIA_TEXTO'] == "Aéreo") & (me2l_15['Dias LT BO'].between(1, 15)),
    (me2l_15['VIA_TEXTO'] == "Aéreo") & (me2l_15['Dias LT BO'].between(16, 30)),
    (me2l_15['VIA_TEXTO'] == "Aéreo") & (me2l_15['Dias LT BO'].between(31, 45)),
    (me2l_15['VIA_TEXTO'] == "Courier") & (me2l_15['Dias LT BO'].between(1, 7)),
    (me2l_15['VIA_TEXTO'] == "Courier") & (me2l_15['Dias LT BO'].between(8, 15)),
    (me2l_15['VIA_TEXTO'] == "Courier") & (me2l_15['Dias LT BO'].between(16, 21)),
    (me2l_15['VIA_TEXTO'] == "Courier") & (me2l_15['Dias LT BO'] > 21),
    (me2l_15['VIA_TEXTO'] == "Aéreo") & (me2l_15['Dias LT BO'] > 45)
]

choices = [
    "De 1 a 30 días",
    "De 31 a 60 días",
    "De 61 a 90 días",
    "De 91 días a más",
    "De 1 a 15 días",
    "De 16 a 30 días",
    "De 31 a 45 días",
    "De 1 a 7 días",
    "De 8 a 15 días",
    "De 16 a 21 días",
    "De 21 días a más",
    "De 45 días a más"
]

# Assign the values based on conditions
me2l_15['Categoría LT'] = np.select(conditions, choices, default="Dentro de LT")



In [321]:
import pandas as pd
import numpy as np

# Assuming CE4 is a column in the DataFrame and BV1 is a variable
BV1 = 45  # Assuming the value of $BV$1 is 45

# Define conditions and choices
conditions = [
    me2l_15['Statur TR No FAct'] == "Replanificar +45",
    me2l_15['Statur TR No FAct'] == "Fecha Teórica",
    (me2l_15['Statur TR No FAct'] == "TR Vencido") & ((me2l_15['Categoría LT'] == 0) | (me2l_15['Categoría LT'] == "Dentro de LT")),
    (me2l_15['Statur TR No FAct'] == "TR Vencido") & (me2l_15['Categoría LT'] == "De 31 a 60 días"),
    (me2l_15['Statur TR No FAct'] == "TR Vencido") & (me2l_15['Categoría LT'] == "De 61 a 90 días")
]

choices = [
    hoy + timedelta(days =45),
    me2l_15['Fecha Facturacion Final'],
    hoy + timedelta(days =30),
    hoy + timedelta(days =60),
    hoy + timedelta(days =90)
]

# Assign the values based on conditions
me2l_15['Fecha No Fact'] = np.select(conditions, choices, default=hoy + timedelta(days =120))


In [322]:
me2l_15['Fecha No Fact'] = pd.to_datetime(me2l_15['Fecha No Fact'])

In [323]:
me2l_15= me2l_15.merge(tabla_final[['AUX','CANTIDAD_BO']], left_on='AUX',right_on='AUX', how='left')

In [324]:
me2l_15['CANTIDAD_BO'] = me2l_15['CANTIDAD_BO']*me2l_15['Corresponde']

In [325]:
me2l_15['CANTIDAD_BO'].fillna(0, inplace=True)

In [326]:
import numpy as np

# Crear una condición basada en la columna 'TIPO'
condicion = (me2l_15['TIPO'] == 'OEM') & (me2l_15['Por calcular (cantidad)'] == 0)

# Actualizar la columna 'Cantidad en BO' basada en la condición
me2l_15['CANTIDAD_BO'] = np.where(condicion, 0, me2l_15['CANTIDAD_BO'])



In [327]:
side_a = me2l_15[['AUX','Status TR Fact',   'Material','Texto breve','Qty Fact Corr','Centro','NomSector_actual','Origen','TIPO','Cl.documento compras','Fecha Facturacion Final','Nombre del proveedor','Grupo de compras']]
side_b = me2l_15[['AUX','Statur TR No FAct','Material','Texto breve','CANTIDAD_BO',     'Centro','NomSector_actual','Origen','TIPO','Cl.documento compras','Fecha No Fact',          'Nombre del proveedor','Grupo de compras']]

In [328]:
side_a.rename(columns={'Status TR Fact':'Status','Qty Fact Corr':'Cantidad','Fecha Facturacion Final':'Fecha'}, inplace=True)
side_b.rename(columns={'Statur TR No FAct':'Status','CANTIDAD_BO':'Cantidad','Fecha No Fact':'Fecha'}, inplace=True)
tr_final = pd.concat([side_a, side_b])

C:\Users\lravlic\AppData\Local\Temp\ipykernel_2020\772410482.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  side_a.rename(columns={'Status TR Fact':'Status','Qty Fact Corr':'Cantidad','Fecha Facturacion Final':'Fecha'}, inplace=True)
C:\Users\lravlic\AppData\Local\Temp\ipykernel_2020\772410482.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  side_b.rename(columns={'Statur TR No FAct':'Status','CANTIDAD_BO':'Cantidad','Fecha No Fact':'Fecha'}, inplace=True)


In [329]:
tr_final.reset_index()

,index,AUX,Status,Material,Texto breve,Cantidad,Centro,NomSector_actual,Origen,TIPO,Cl.documento compras,Fecha,Nombre del proveedor,Grupo de compras
0,0,420000494110,No Facturado,PORTADOCSUZUKI,AXS.PORTA DOCUMENTOS SUZUKI,0.0,201.0,Accesorios,CL,AXS,ZSTO,2024-04-05 00:00:00.000000,10001447 I.V. Fabrica de producto,RR2
1,1,420000494120,No Facturado,PORTADOCCHANGAN,AXS.PORTA DOCUMENTOS CHANGAN COMERCIAL,0.0,201.0,Accesorios,CL,AXS,ZSTO,2024-04-05 00:00:00.000000,10001447 I.V. Fabrica de producto,RR2
2,2,420000494210,No Facturado,ADHWARNING,AXS.ADHESIVO WARNING CX-5,0.0,201.0,Accesorios,CL,AXS,ZSTO,2024-04-05 00:00:00.000000,10001669 Produccion Y Publicitari,RR2
3,3,420000494310,No Facturado,CANDRUEDAPOER,AXS.CANDADO RUEDA POER,0.0,201.0,Accesorios,CL,AXS,ZSTO,2024-04-05 00:00:00.000000,10002739 Victor Paredes E Hijos L,RR2
4,4,420000494320,No Facturado,CANDRUEDANEWBT50,AXS.CANDADO RUEDA REPUESTO NEW BT50,0.0,201.0,Accesorios,CL,AXS,ZSTO,2024-04-05 00:00:00.000000,10002739 Victor Paredes E Hijos L,RR2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31247,15621,4300014353270,Replanificar +45,5512200U2260,GUARDAFANGO DEL DER [NEW],1.0,201.0,JAC Cars,PE,OEM,ZVOR,2024-05-10 09:33:33.601399,PU02 DERCO PERU S.A.,RR9
31248,15622,4300014353280,Replanificar +45,6102200U19C0,TAPIZ PUERTA DEL DER,1.0,201.0,JAC Cars,PE,OEM,ZVOR,2024-05-10 09:33:33.601399,PU02 DERCO PERU S.A.,RR9
31249,15623,4300014353290,Replanificar +45,8103010P306AA,COMPRESOR AC,1.0,201.0,JAC Cars,PE,OEM,ZVOR,2024-05-10 09:33:33.601399,PU02 DERCO PERU S.A.,RR9
31250,15624,4300014353300,Replanificar +45,8400500R001E,PANEL DELT,1.0,201.0,JAC Cars,PE,OEM,ZVOR,2024-05-10 09:33:33.601399,PU02 DERCO PERU S.A.,RR9


In [330]:
tr_final.to_excel('TR FINAL PRUEBA R3.xlsx')

In [331]:
fin = time.time()

In [332]:
total = fin - inicio

In [333]:
print(f'Tiempo total de ejecucion: {round(total/60, 2)} minutos'  )

Tiempo total de ejecucion: 3.73 minutos
